***
<center><img src='https://github.com/HesamAsad/HesamAsad.github.io/blob/main/images/LOGO_EN_blue.png?raw=true' width=250/></center>

# <center>
## <center><span style='font-family:"Times New Roman"'><span style='background:dark'>  <font color='#1966ab'>$\LARGE{\text{Sharif University of Technology}}$</font>  <br>  <br>  <font color='#FFFFCC'>$\Large{\text{Masoud Tahmasbi}}$ </font> <br> <font color='#FFFFCC'>$\normalsize {\text{Student ID: } 402200275}$ </font>  <br>  <br> <font color='#33FFCC'>$\large{\text{EE577: Massive Data Analytics}}$</font> <br>  <font color='#FFFF00'>$\normalsize{\text{Assignment #4 - Random Walks based Recommender System}}$ </font>
<br>

***

# $\text{Data Preparation}$

In [ ]:
! pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp '/content/gdrive/My Drive/twitter_data_v2.tar.gz' twitter_data
!tar -xzvf "/content/twitter_data"

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW4") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

In [ ]:
twitter_rdd = sc.textFile("twitter_data_v2.jsonl")

In [ ]:
import json
parsed_rdd = twitter_rdd.map(lambda x: json.loads(x))
parsed_rdd.count()

4175860

In [ ]:
parsed_rdd.takeSample(False,1,112)

[{'in_reply_to_status_id_str': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_status_id': None,
  'in_reply_to_user_id': None,
  'in_reply_to_screen_name': None,
  'truncated': False,
  'is_quote_status': False,
  'retweet_count': 20,
  'reply_count': 0,
  'quote_count': 0,
  'favorite_count': 0,
  'favorited': False,
  'retweeted': True,
  'possibly_sensitive': None,
  'lang': 'fa',
  'geo': None,
  'view_count': 0,
  'tweet_type': 'retweeted',
  'emojis': None,
  'text': 'فک کن بچه رو کشتن، بچه ۲۰ ساله، حالام میخوان مادرش رو ۵ سال ببرن زندان.\nواقعا آخوند چه کثافتی هست که این بی پدرا انداختن تو زندگی ما.\n#محمدجواد_زاهدی\n#مهسا_یزدانی',
  'user': {'id': '1565039849287172096',
   'id_str': '1565039849287172096',
   'created_at': 1661969566,
   'name': 'Negin',
   'screen_name': 'Negin88419550',
   'location': '',
   'description': '',
   'verified': False,
   'followers_count': 20,
   'friends_count': 4,
   'listed_count': 0,
   'favourites_count': 15093,
   'statuses_count':

***
# $\text{Creating Users' Graph}$

In [ ]:
def users_tweets_extractor(tweet):
    if tweet.get('tweet_type') == 'generated':
        try:
            generator_uid = tweet.get('user').get('id')
        except: return []
        try:
            generated_tweet_id = tweet.get('id')
        except: return []

        return [(generator_uid,[generated_tweet_id])]

    if tweet.get('tweet_type') == 'quoted':
        try:
            quoting_uid = tweet.get('user').get('id')
        except: return []
        try:
            quoted_uid = tweet.get('quoted_status').get('user').get('id')
        except: return []
        try:
            quoted_tweet_id = tweet.get('quoted_status').get('id')
        except: return []

        return [(quoted_uid,[quoted_tweet_id]) , (quoting_uid,[quoted_tweet_id])]


    if tweet.get('tweet_type') == 'replied':
        try:
            replying_uid = tweet.get('user').get('id')
        except: return []
        try:
            replied_uid = tweet.get('in_reply_to_user_id_str')
        except: return []
        try:
            replied_tweet_id = tweet.get('in_reply_to_status_id_str')
        except: return []

        return [(replied_uid,[replied_tweet_id]) , (replying_uid,[replied_tweet_id])]


    if tweet.get('tweet_type') == 'retweeted':
        try:
            retweeting_uid = tweet.get('user').get('id')
        except: return []
        try:
            retweeted_uid = tweet.get('retweeted_status').get('user').get('id')
        except: return []
        try:
            retweeted_tweet_id = tweet.get('retweeted_status').get('id')
        except: return []

        return [(retweeted_uid,[retweeted_tweet_id]) , (retweeting_uid,[retweeted_tweet_id])]


In [ ]:
users_rdd_flat = parsed_rdd.flatMap(lambda x:users_tweets_extractor(x))
users_rdd_flat.count()

7355551

In [ ]:
users_rdd = users_rdd_flat.reduceByKey(lambda a,b: a+b)
users_rdd.count()

330500

In [ ]:
# users_rdd.first()

In [ ]:
def tweets_related_users(tweet):
    if tweet.get('tweet_type') == 'generated':
        try:
            generator_uid = tweet.get('user').get('id')
        except: return []
        try:
            generated_tweet_id = tweet.get('id')
        except: return []

        return [(generated_tweet_id,[generator_uid])]

    if tweet.get('tweet_type') == 'quoted':
        try:
            quoting_uid = tweet.get('user').get('id')
        except: return []
        try:
            quoted_uid = tweet.get('quoted_status').get('user').get('id')
        except: return []
        try:
            quoted_tweet_id = tweet.get('quoted_status').get('id')
        except: return []

        return [(quoted_tweet_id,[quoted_uid]) , (quoted_tweet_id,[quoting_uid])]


    if tweet.get('tweet_type') == 'replied':
        try:
            replying_uid = tweet.get('user').get('id')
        except: return []
        try:
            replied_uid = tweet.get('in_reply_to_user_id_str')
        except: return []
        try:
            replied_tweet_id = tweet.get('in_reply_to_status_id_str')
        except: return []

        return [(replied_tweet_id,[replied_uid]) , (replied_tweet_id,[replying_uid])]



    if tweet.get('tweet_type') == 'retweeted':
        try:
            retweeting_uid = tweet.get('user').get('id')
        except: return []
        try:
            retweeted_uid = tweet.get('retweeted_status').get('user').get('id')
        except: return []
        try:
            retweeted_tweet_id = tweet.get('retweeted_status').get('id')
        except: return []

        return [(retweeted_tweet_id,[retweeted_uid]) , (retweeted_tweet_id,[retweeting_uid])]


In [ ]:
tweets_rdd_flat = parsed_rdd.flatMap(lambda x:tweets_related_users(x))
tweets_rdd_flat.count()

7355551

In [ ]:
tweets_rdd = tweets_rdd_flat.reduceByKey(lambda a,b: a+b)
tweets_rdd.count()

1475790

In [ ]:
# tweets_rdd.first()

***
# $\text{User #1}$

In [ ]:
target_user = ('1697206791715635200',
               ['1736250426230493213','1736250178363883872','1736249928018764235',
                '1735951927584682444','1736108738921033809','1736287140860829901',
                '1736622445505741159','1734655981412573264','1734657820010991962',
                '1734661707908120774','1734661220601352349','1734659777710498122',
                '1734658201302577473','1734656185226424457','1734799518305034308',
                '1734798971577536976','1734798971577536976','1734798173476950425',
                '1734826498190717030','1734531657623859526','1734526333823176854',
                '1734555309257982198','1734830893259239623','1736287052885336178',
                '1734613998107086944','1734568158986375391','1734545449514582130',
                '1734591710326370694','1734591032791060488','1734876907928264981',
                '1734589323377352980','1734887227686047878','1734588341004181625',
                '1734589532039741549','1734589532039741549','1734830893259239623',
                '1734813269620048114','1734590919716765976','1734541649462649082',
                '1734588709524136318','1734613510997397685','1734613510997397685',
                '1734549000622051631','1734579905252209035','1734536728877154500',
                '1734540761868202400','1734540640837419434','1734562848087388436',
                '1736345546997088650','1733927496456794435','1733927496456794435',
                '1734918783771250916','1734998874064912711','1735000541858504730',
                '1735003971243192520','1736076770128851061','1736046266474262648',
                '1735995782283555029','1736079804095517131','1733027686677717325',
                '1736023600539328584','1736068818160533903','1736068403750695299',
                '1735262181023695354','1735731570865000582'])

In [ ]:
target_user_tweets_ids = list(set([x for x in target_user[1]]))
target_user_tweets_ids

['1734613510997397685',
 '1734590919716765976',
 '1734555309257982198',
 '1734655981412573264',
 '1735731570865000582',
 '1736287140860829901',
 '1734549000622051631',
 '1734799518305034308',
 '1734661220601352349',
 '1736250178363883872',
 '1734589323377352980',
 '1735003971243192520',
 '1736076770128851061',
 '1733027686677717325',
 '1734656185226424457',
 '1734661707908120774',
 '1734562848087388436',
 '1735951927584682444',
 '1734813269620048114',
 '1734830893259239623',
 '1734588341004181625',
 '1734588709524136318',
 '1734531657623859526',
 '1734568158986375391',
 '1734658201302577473',
 '1734826498190717030',
 '1736068818160533903',
 '1735262181023695354',
 '1735995782283555029',
 '1736108738921033809',
 '1734887227686047878',
 '1734545449514582130',
 '1734541649462649082',
 '1734540761868202400',
 '1734657820010991962',
 '1734591032791060488',
 '1734589532039741549',
 '1734798971577536976',
 '1734536728877154500',
 '1736249928018764235',
 '1734659777710498122',
 '17345406408374

In [ ]:
def users_from_tweets(user_tweets_ids, tweets_rdd):
    if tweets_rdd[0] in user_tweets_ids:
        return list(set(tweets_rdd[1])) # Unique
    else:
        return []

In [ ]:
def tweets_from_users(user_ids, users_rdd):
    if users_rdd[0] in user_ids:
        return list(set(users_rdd[1])) # Unique
    else:
        return []

In [ ]:
_xhop_users = tweets_rdd.flatMap(lambda x: users_from_tweets(target_user_tweets_ids,x))
_1hop_users = _xhop_users.collect()
# _1hop_users

In [ ]:
_xhop_tweets = users_rdd.flatMap(lambda x: tweets_from_users(_1hop_users,x))
_2hop_tweets = _xhop_tweets.collect()
# _2hop_tweets

In [ ]:
_xhop_users = tweets_rdd.flatMap(lambda x: users_from_tweets(_2hop_tweets,x))
_3hop_users = _xhop_users.collect()
# _3hop_users

In [ ]:
_xhop_tweets = users_rdd.flatMap(lambda x: tweets_from_users(_3hop_users,x))
_4hop_tweets = _xhop_tweets.collect()
# _4hop_tweets

In [ ]:
all_related_users = _1hop_users
# all_related_users = _1hop_users + _3hop_users
len(all_related_users)

2277

In [ ]:
all_related_tweets = _2hop_tweets
# all_related_tweets = _2hop_tweets + _4hop_tweets
len(all_related_tweets)

109440

In [ ]:
def users_with_tweets_generator(all_related_users,user):
    if user[0] in all_related_users:
        return [(user[0],user[1])]
    else:
        return []

In [ ]:
def tweets_with_users_generator(all_related_tweets,tweet):
    if tweet[0] in all_related_tweets:
        return [(tweet[0],tweet[1])]
    else:
        return []

In [ ]:
users_for_graph_rdd = users_rdd.flatMap(lambda x: users_with_tweets_generator(all_related_users,x))
users_for_graph = users_for_graph_rdd.collect()

In [ ]:
tweets_for_graph_rdd = tweets_rdd.flatMap(lambda x:tweets_with_users_generator(all_related_tweets,x))
tweets_for_graph = tweets_for_graph_rdd.collect()

In [ ]:
len(users_for_graph)

1891

In [ ]:
len(tweets_for_graph)

46024

In [ ]:
import random
def random_walk(target_user,users_for_graph,tweets_for_graph,return_prob = 0.8,teleport = False):
    seen_users = []
    current_user_tweets = target_user[1]
    for i in range(1,10000):
        if len(current_user_tweets) == 0:
            current_user_tweets = target_user[1]
            continue

        current_tweet_id = random.choice(current_user_tweets)
        idx = next((i for i, v in enumerate(tweets_for_graph) if v[0] == current_tweet_id), None)
        if idx == None:
            current_user_tweets = target_user[1]
            continue

        current_user_id = random.choice(tweets_for_graph[idx][1])
        seen_users.append((current_user_id,1))
        idx = next((i for i, v in enumerate(users_for_graph) if v[0] == current_user_id), None)
        if idx == None:
            current_user_tweets = target_user[1]
            continue
        current_user_tweets = random.choice(users_for_graph[idx][1])

        if teleport and random.random() < return_prob:
            current_user_tweets = target_user[1]

    return seen_users

In [ ]:
seen_users_all = random_walk(target_user,users_for_graph,tweets_for_graph,return_prob = 0.8)

In [ ]:
from itertools import groupby
from operator import itemgetter

first = itemgetter(0)
seen_users_cnt = [(k, sum(item[1] for item in tups_to_sum))
        for k, tups_to_sum in groupby(sorted(seen_users_all, key=first), key=first)]

In [ ]:
seen_users_sorted = sorted(seen_users_cnt, key=lambda x: x[1], reverse=True)
top_users = seen_users_sorted[1:20]
top_users

[('1654179648761102337', 420),
 ('1710302619371982848', 308),
 ('1487097029990002688', 205),
 ('1648401324608827392', 166),
 ('1593669637102866432', 124),
 ('1226113547542773762', 122),
 ('1696453872074539008', 87),
 ('1253707166919397377', 85),
 ('1724483603600486400', 81),
 ('1632351425383522312', 80),
 ('1714031594598490113', 78),
 ('1398658879827886085', 73),
 ('1710903076574359552', 71),
 ('1221434210927464448', 70),
 ('1305617097339400192', 68),
 ('1654409094076268546', 61),
 ('1549397580869664775', 56),
 ('1643233712916578309', 54),
 ('1698191107786518528', 53)]

In [ ]:
idx = next((i for i, v in enumerate(users_for_graph) if v[0] == top_users[1][0]), None)
sample_tweets = users_for_graph[idx][1]

In [ ]:
def tweet_parser(tweet_ids,tweet):
    if tweet.get('id') in tweet_ids:
        return [(tweet.get('text'),)]
    else:
        return []

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(sample_tweets,x))
recommended_tweets.collect()

[('نام : حمید نوری\nوضعیت : ساکن سلول انفرادی\nجرم : نامعلوم\nشاکی : سازمان تروریستی و ضد بشر منافقین\nقاضی : مدعیان دفاع از حقوق بشر\nمحل صدور حکم : کشور حامی سوزاندن قرآن\nحکم : حبس ابد\n\n#freehamidnouri https://t.co/VQFx0msvzs',),
 ('و چقدر بوی عطر شهیدان ، \n\nهمه جا پیچیده  ...\n\n#گمنام_مثل_مادر \n#لشکر_علویان',),
 ('@s_isatahmasbi تا ابد مدیون قهرمانان واقعی وطن هستیم💔\n#گمنام_مثل_مادر\n#لشکر_علویان',),
 ('شهدا میراث دار  غیرت علوی و نجابت فاطمی هستند\n#گمنام_مثل_مادر\n#لشکر_علویان https://t.co/obFOLyICtP',),
 ('وصیت نامه یکی از  شهدای بزرگوار :  خدایا نشود که در زندگی دو دستی به درخت زندگی بچسبم و همچون میوه ایی پوسیده بیافتم خدایا تا ایمانی هست مرا برای خود قربانی کن\n#گمنام_مثل_مادر\n#لشکر_علویان https://t.co/CzBw8KoKmM',),
 ('زندگی پر از انتخاب هایی است که مسیر ها رو عوض می\u200cکنند، اینکه قهرمان زندگی چه شخصی باشه در عاقبت بخیری خیلی مهمه\n#گمنام_مثل_مادر\n#لشکر_علویان https://t.co/QGEkchpZl4',),
 ('چراغ راه\n#گمنام_مثل_مادر\n#لشکر_علویان https://t.co/XNvjz1TJ01',),
 ('رو

In [ ]:
idx = next((i for i, v in enumerate(users_for_graph) if v[0] == top_users[2][0]), None)
sample_tweets = users_for_graph[idx][1]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(sample_tweets,x))
recommended_tweets.collect()

[('و چقدر بوی عطر شهیدان ، \n\nهمه جا پیچیده  ...\n\n#گمنام_مثل_مادر \n#لشکر_علویان',),
 ('@s_isatahmasbi تا ابد مدیون قهرمانان واقعی وطن هستیم💔\n#گمنام_مثل_مادر\n#لشکر_علویان',),
 ('وصیت نامه یکی از  شهدای بزرگوار :  خدایا نشود که در زندگی دو دستی به درخت زندگی بچسبم و همچون میوه ایی پوسیده بیافتم خدایا تا ایمانی هست مرا برای خود قربانی کن\n#گمنام_مثل_مادر\n#لشکر_علویان https://t.co/CzBw8KoKmM',),
 ('زندگی پر از انتخاب هایی است که مسیر ها رو عوض می\u200cکنند، اینکه قهرمان زندگی چه شخصی باشه در عاقبت بخیری خیلی مهمه\n#گمنام_مثل_مادر\n#لشکر_علویان https://t.co/QGEkchpZl4',),
 ('لاله های زهرایی\n#وداع_با_لاله\u200cها\n#گمنام_مثل_مادر https://t.co/MVVre3iDnY',),
 ('حضور مردم قدر شناس دیار علویان در مراسم وداع با لاله\u200cها\n#وداع_با_لاله\u200cها\n#گمنام_مثل_مادر https://t.co/JWvgiokZzn',),
 ('دیار علویان و مردم همیشه در صحنه\n#وداع_با_لاله\u200cها\n#گمنام_مثل_مادر https://t.co/1NNoFp1AGe',),
 ('در پی اسم و...\n#گمنام_مثل_مادر\n#وداع_با_لاله\u200cها https://t.co/QgF5gIXki9',),
 ('روضه بی

***
# $\text{User #2}$

In [ ]:
target_user = ('4702453171',
 [('4702453171', '1734001719539810691', 3),
  ('4702453171', '1734471154646049090', 3),
  ('4702453171', '1734471452450005121', 3),
  ('1653467502972682270', '1734079495177461916', 2),
  ('25710094', '1734490763394798040', 2),
  ('60171166', '1736513161631588535', 2),
  ('17532913', '1734073641908981882', 2),
  ('4702453171', '1733813004280459569', 2),
  ('4702453171', '1734086631366311958', 3),
  ('4702453171', '1734086631366311958', 2),
  ('17532913', '1734087759646871761', 2),
  ('4702453171', '1734096096526291087', 3),
  ('4702453171', '1734099408688488815', 3),
  ('4702453171', '1734099878626705917', 3),
  ('4702453171', '1734100866527817896', 3),
  ('4702453171', '1735916939380212209', 3),
  ('4702453171', '1734103110279102546', 3),
  ('4702453171', '1726253167547924649', 2),
  ('15090494', '1734104725874713032', 2),
  ('360461731', '1734106135055958309', 2),
  ('1271067471760568322', '1734105477301117383', 2),
  ('4702453171', '1734108334402228647', 3),
  ('767126934', '1734108466426380588', 2),
  ('302678816', '1736273645201182727', 2),
  ('4702453171', '1736290273133490433', 3),
  ('302678816', '1736289193905594796', 2),
  ('767126934', '1734118849220374966', 2),
  ('712752292194357249', '1735382946834518426', 2),
  ('767126934', '1734119784206172308', 2),
  ('701771232', '1735211967982272607', 2),
  ('1321668098076954624', '1735938271702564881', 2),
  ('824950806411624448', '1735580501056819243', 2),
  ('824950806411624448', '1735580764895334548', 2),
  ('4702453171', '1736308051324387667', 3),
  ('1734473657940946944', '1735583061285179397', 2),
  ('4702453171', '1734134990097518886', 3),
  ('1676645415758733333', '1734136964343898342', 2),
  ('4702453171', '1735580419205091753', 2),
  ('4702453171', '1735951311202316406', 3),
  ('1271067471760568322', '1736310685238268342', 2),
  ('4702453171', '1735951311202316406', 2),
  ('1141720339', '1735953233720246476', 2),
  ('4702453171', '1734142887552549260', 3),
  ('1676645415758733333', '1734139635738009661', 2),
  ('4702453171', '1736315796605817179', 3),
  ('60171166', '1736686396276891799', 2),
  ('4702453171', '1735956104247484746', 3),
  ('4702453171', '1735960050726842606', 3),
  ('4702453171', '1734151214852444359', 3),
  ('4702453171', '1735968721225605288', 3),
  ('3140575865', '1735974424472432879', 2),
  ('597673178', '1734950125850833149', 2),
  ('4702453171', '1734164069148492141', 3),
  ('1322110382455087106', '1735979725829419189', 2),
  ('4702453171', '1734638728558362920', 3),
  ('4702453171', '1735235674977034592', 2),
  ('840241125499777024', '1734645197139497160', 2),
  ('4702453171', '1734179665147805779', 3),
  ('1639263584407416839', '1734179803899555964', 2),
  ('1639263584407416839', '1734180929269023060', 2),
  ('60171166', '1734180970050310521', 2),
  ('1387366298099798025', '1735263247018651678', 2),
  ('855095592', '1736351208460194151', 2),
  ('4702453171', '1734931036251869266', 3),
  ('1062546499', '1736309770812793130', 1),
  ('855095592', '1736353904416198814', 2),
  ('4702453171', '1735268263754354725', 3),
  ('1010561817997922308', '1735268155583267124', 2),
  ('1062546499', '1736309770812793130', 2),
  ('4702453171', '1735255090083078151', 2),
  ('4702453171', '1736363003379884521', 3),
  ('4702453171', '1736009369383243808', 3),
  ('855095592', '1736363383593578831', 2),
  ('4702453171', '1735968721225605288', 2),
  ('302678816', '1736366887057887492', 2),
  ('4702453171', '1736013977610592680', 3),
  ('4702453171', '1736014753456177493', 3),
  ('1231450050389315585', '1736016324734714299', 2),
  ('824950806411624448', '1736016158841532843', 2),
  ('824950806411624448', '1736018543064367317', 2),
  ('1567482915600142339', '1735285273020203467', 2),
  ('1206642366830002179', '1734220953184268661', 2),
  ('46853901', '1734226095593976099', 2),
  ('1152293110212263937', '1734212837583507605', 1),
  ('4702453171', '1735668040014979319', 3),
  ('1576644054699642880', '1736778687595336173', 2),
  ('4702453171', '1734230963670986760', 3),
  ('4702453171', '1734694077206254058', 2),
  ('1576644054699642880', '1736783279682998688', 2),
  ('1562505373285658631', '1735327273933267052', 2),
  ('1562505373285658631', '1735327920371958034', 2),
  ('1677469467637342209', '1736415934116507919', 2),
  ('4702453171', '1735002509410762777', 3),
  ('4702453171', '1735002509410762777', 2),
  ('1562505373285658631', '1735336112640442651', 2),
  ('732854166', '1735699878984982796', 2),
  ('2685820495', '1734262222023545230', 2),
  ('4702453171', '1735013203237617714', 3),
  ('240631009', '1736041432056578393', 2),
  ('4702453171', '1736086585462903227', 3),
  ('4702453171', '1736087319415787587', 3),
  ('302678816', '1735022147720859827', 2),
  ('3304332625', '1735022541943587001', 2),
  ('4702453171', '1734289632567689542', 3),
  ('1556954568797392897', '1735336832911438191', 2),
  ('4702453171', '1735360865161904553', 3),
  ('1556954568797392897', '1735363164877869514', 2),
  ('4702453171', '1736487283690233926', 3),
  ('4702453171', '1736491571539775817', 3),
  ('824950806411624448', '1736493528803315964', 2)])

In [ ]:
target_user = (target_user[0],list(set([x[1] for x in target_user[1]])))
target_user

In [ ]:
target_user_tweets_ids = target_user[1]
target_user_tweets_ids

In [ ]:
_xhop_users = tweets_rdd.flatMap(lambda x: users_from_tweets(target_user_tweets_ids,x))
_1hop_users = _xhop_users.collect()
# _1hop_users

In [ ]:
_xhop_tweets = users_rdd.flatMap(lambda x: tweets_from_users(_1hop_users,x))
_2hop_tweets = _xhop_tweets.collect()
# _2hop_tweets

In [ ]:
_xhop_users = tweets_rdd.flatMap(lambda x: users_from_tweets(_2hop_tweets,x))
_3hop_users = _xhop_users.collect()
# _3hop_users

In [ ]:
_xhop_tweets = users_rdd.flatMap(lambda x: tweets_from_users(_3hop_users,x))
_4hop_tweets = _xhop_tweets.collect()
# _4hop_tweets

In [ ]:
all_related_users = _1hop_users
# all_related_users = _1hop_users + _3hop_users
len(all_related_users)

274

In [ ]:
all_related_tweets = _2hop_tweets
# all_related_tweets = _2hop_tweets + _4hop_tweets
len(all_related_tweets)

5482

In [ ]:
users_for_graph_rdd = users_rdd.flatMap(lambda x: users_with_tweets_generator(all_related_users,x))
users_for_graph = users_for_graph_rdd.collect()
# users_for_graph

In [ ]:
tweets_for_graph_rdd = tweets_rdd.flatMap(lambda x:tweets_with_users_generator(all_related_tweets,x))
tweets_for_graph = tweets_for_graph_rdd.collect()

In [ ]:
len(users_for_graph)

128

In [ ]:
len(tweets_for_graph)

5038

In [ ]:
seen_users_all = random_walk(target_user,users_for_graph,tweets_for_graph,return_prob = 0.8)

In [ ]:
from itertools import groupby
from operator import itemgetter

first = itemgetter(0)
seen_users_cnt = [(k, sum(item[1] for item in tups_to_sum))
        for k, tups_to_sum in groupby(sorted(seen_users_all, key=first), key=first)]

In [ ]:
seen_users_sorted = sorted(seen_users_cnt, key=lambda x: x[1], reverse=True)
top_users = seen_users_sorted[1:20]
top_users

[('824950806411624448', 127),
 ('302678816', 116),
 ('1562505373285658631', 77),
 ('855095592', 68),
 ('60171166', 67),
 ('767126934', 67),
 ('1271067471760568322', 60),
 ('1556954568797392897', 57),
 ('1576644054699642880', 52),
 ('1639263584407416839', 49),
 ('17532913', 44),
 ('1676645415758733333', 38),
 ('597673178', 38),
 ('1010561817997922308', 37),
 ('840241125499777024', 37),
 ('1321668098076954624', 36),
 ('46853901', 36),
 ('1231450050389315585', 34),
 ('1567482915600142339', 34)]

In [ ]:
idx = next((i for i, v in enumerate(users_for_graph) if v[0] == top_users[1][0]), None)
sample_tweets = users_for_graph[idx][1]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(sample_tweets,x))
recommended_tweets.collect()

[('دانشگاه MIT هم جای عجیبیه\u200c\n\nتو پیج اعترافات MIT، یه نفر اومده اعتراف کرده یه دختر (احتمالا کراش طرف بوده) اونو آنفالو کرده بعد از اینکه فهمیده مقالش ریجکت شده 😂\nحقیقتا این یکی خیلی دارک بود. مثلا طرف کات کنه با دوست پسرش بهش بگه مقالت ریجکت شده من دیگه نمیتونم باهات باشم\n\n#فقر_فساد_گرونی',),
 ('عالیه:)) https://t.co/747xf8YIx9',),
 ('این کیف پول هم چقدر کارمزد میگیره. هر کاری میکنی یه درصد میگیره. \nالان میگه کارت رو بخوای شارژ کنی باید تتر بگیری. هر تتر بگیری 8 % بر میدارم کارمزد.\nهر تتر هم 23 لیر زده.\nیعنی شما 100 تتر بگیری. \n92 تتر میده بهت \nمیشه 2116 لیر',),
 ('@hkazazi اخه حتی قبولم نمیکنه لیر رو از جای دیگه بگیری بزنی بهش :/\nمن حواله میزدم اصلا خیلی ارزون تر',),
 ('یکی از این پراپا تقسیم سود صد درصدی گذاشته:) بابا اینقدر بلند اعلام اسکم نکن.',),
 ('پراگ اسکان زیباست\nهر سری میام اینجا لذت میبرم https://t.co/VVVEl8NYGo',),
 ('@SeyedAllyn فکر نکنم اسکم باشه ها . فقط 100 ساعت و فقط برای اولین خرید گذاشته . \nبیشتر یاد تخفیف های سنگین ریحون میوفتم که جذب مخاطب کنه',

In [ ]:
idx = next((i for i, v in enumerate(users_for_graph) if v[0] == top_users[2][0]), None)
sample_tweets = users_for_graph[idx][1]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(sample_tweets,x))
recommended_tweets.collect()

[('دوستان پلاس پلی استیشن از کجا بگیرم قیمتش خوب باشه؟',),
 ('دوستان من بدنبال کار بصورت پروژه ای هستم(پایتون)\nممنون میشم ریتوییت کنید🙏\n#ریتوییت',),
 ('البته الان اسپاتیفای پلن ۳ ماهه پرمیوم رایگان هم داره.\nمیتونید کارت فیک هم بزنید و اگه قبول شد استفادش کنید.\n\nهمین الان برای دوستم با کارت فیک گرفتم :)))) https://t.co/QEw09pML8u',),
 ('به نظرتون جواب همچین متنی چیه؟ https://t.co/Hy74QC5m9G',),
 ('@nimbit__ من ازشون زیاد خرید داشتم و برای من کاملا اوکی بوده تا الان. قیمت هاشونم کم بوده به نسبت بقیه جاها.\nولییییی\nاین دلیل خوبی نیست. من هیچی تضمین نمیکنم 😬',),
 ('@Nikolaona نه فقط میخواستم بدونم تا حالا ازشون خرید داشتی یا نه:)\nبازم ممنون\nفقط با مستر کارت ترکیه باید vpn ترکیه هم بزنیم دیگه؟\nچون ریجن اسپاتیفای هم باید عوض بشه فک کنم',),
 ('@nimbit__ نه نیاز نیس\nموقع زدن شماره کارت از پایین کشور رو بکن ترکیه',),
 ('@Nikolaona بعد یه سوال دیگه\nبعد بخوام با همین ریجن گوگل پلی عوض کنم میشه؟\nمثلا همین مستر کارت رو اد کنم گوگل پلی ریجنم بشه ترکیه(تا حالا نکردم)',),
 ('از من به شما ن

***
# $\text{User #3}$

In [ ]:
target_user = ('1693973461306109952',
 [('1574904470835798041', '1734442494039826873', 2),
  ('1574904470835798041', '1734883099647914204', 2),
  ('1574904470835798041', '1735771172472492519', 2),
  ('1693973461306109952', '1736853602969280923', 3),
  ('1693973461306109952', '1736855151917334935', 3),
  ('1693973461306109952', '1735069630308266029', 3),
  ('1693973461306109952', '1735401606160785610', 3),
  ('1693973461306109952', '1735071498032058504', 3),
  ('1693973461306109952', '1735071498032058504', 2),
  ('1574904470835798041', '1735403820757426674', 2)])

In [ ]:
target_user = (target_user[0],list(set([x[1] for x in target_user[1]])))
target_user

('1693973461306109952',
 ['1735771172472492519',
  '1736855151917334935',
  '1735403820757426674',
  '1734883099647914204',
  '1735069630308266029',
  '1736853602969280923',
  '1735401606160785610',
  '1735071498032058504',
  '1734442494039826873'])

In [ ]:
target_user_tweets_ids = target_user[1]
target_user_tweets_ids

['1735771172472492519',
 '1736855151917334935',
 '1735403820757426674',
 '1734883099647914204',
 '1735069630308266029',
 '1736853602969280923',
 '1735401606160785610',
 '1735071498032058504',
 '1734442494039826873']

In [ ]:
_xhop_users = tweets_rdd.flatMap(lambda x: users_from_tweets(target_user_tweets_ids,x))
_1hop_users = _xhop_users.collect()
# _1hop_users

In [ ]:
_xhop_tweets = users_rdd.flatMap(lambda x: tweets_from_users(_1hop_users,x))
_2hop_tweets = _xhop_tweets.collect()
# _2hop_tweets

In [ ]:
_xhop_users = tweets_rdd.flatMap(lambda x: users_from_tweets(_2hop_tweets,x))
_3hop_users = _xhop_users.collect()
# _3hop_users

In [ ]:
_xhop_tweets = users_rdd.flatMap(lambda x: tweets_from_users(_3hop_users,x))
_4hop_tweets = _xhop_tweets.collect()
# _4hop_tweets

In [ ]:
all_related_users = _1hop_users
# all_related_users = _1hop_users + _3hop_users
len(all_related_users)

13

In [ ]:
all_related_tweets = _2hop_tweets
# all_related_tweets = _2hop_tweets + _4hop_tweets
len(all_related_tweets)

20

In [ ]:
users_for_graph_rdd = users_rdd.flatMap(lambda x: users_with_tweets_generator(all_related_users,x))
users_for_graph = users_for_graph_rdd.collect()
# users_for_graph

In [ ]:
tweets_for_graph_rdd = tweets_rdd.flatMap(lambda x:tweets_with_users_generator(all_related_tweets,x))
tweets_for_graph = tweets_for_graph_rdd.collect()

In [ ]:
len(users_for_graph)

2

In [ ]:
len(tweets_for_graph)

16

In [ ]:
seen_users_all = random_walk(target_user,users_for_graph,tweets_for_graph,return_prob = 0.8)

In [ ]:
from itertools import groupby
from operator import itemgetter

first = itemgetter(0)
seen_users_cnt = [(k, sum(item[1] for item in tups_to_sum))
        for k, tups_to_sum in groupby(sorted(seen_users_all, key=first), key=first)]

In [ ]:
seen_users_sorted = sorted(seen_users_cnt, key=lambda x: x[1], reverse=True)
top_users = seen_users_sorted[1:20]
top_users

[('1574904470835798041', 1193)]

In [ ]:
idx = next((i for i, v in enumerate(users_for_graph) if v[0] == top_users[0][0]), None)
sample_tweets = users_for_graph[idx][1]

In [ ]:
def tweet_parser(tweet_ids,tweet):
    if tweet.get('id') in tweet_ids:
        return [(tweet.get('text'),)]
    else:
        return []

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(sample_tweets,x))
recommended_tweets.collect()

[('تو\nآنجایی\nو \n"آنجا" \nنمی داند \nکه\nچقدر\nخوشبخت\nاست!',),
 ('بخیر ترین صبح قطعا اونی که \nکنار کسی که دوسش داری شروع میشه…',),
 ('آنجا که تو نیستی، کجا خواهد بود…',),
 ('گمان میکنی پایان است ،\nو خدا همه چیز را درست میکند…',),
 ('از بوی تنت فهمیدم\nشاید خیلی دیر رسیدم خیلی دیر \nاما مگر قانون این نبود که هر آنچه دیر می آید\nعاقبت روزی به خانه ی ما خواهد رسید؟…',),
 ('بغلم کن همه جا\nشهر حسودی بکند…',),
 ('ای دیدن تو دین من\nای روی تو ایمان من\nای هست تو پنهان شده\nدر هستی پنهان من…',),
 ('کار می کنم و به تو فکر میکنم\nبه خانه برمی گردم وبه تو فکر میکنم \nبه آن زن تلفن میکنم و درعین حال به تو فکر میکنم \nبه او می گویم حالت چطور است؟ و به تو فکر میکنم\nکجا برویم ؟وبه تو فکر میکنم \nبه او لبخند می زنم ،چشمانم را می بندم و به تو فکر می کنم...',)]

***
# $\text{User #4}$

In [ ]:
target_user = users_rdd.takeSample(False,1,487)[0]
target_user

('1343961231166734336',
 ['1736503973559906442',
  '1736422396721062257',
  '1736384901635539042',
  '1736407359461704039',
  '1736455332614877486',
  '1736307951638323471',
  '1736442052454310094'])

In [ ]:
target_user_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(target_user_tweets_ids,x))
target_user_tweets.collect()

[('مردم گیلان شهید حسن بابانیا را در روز شهادت حضرت زهرا سلام\u200cالله در میان اشک و آه بدرقه کردند.\n#شهدای_فاطمیه\n#انتقام_سخت https://t.co/AXAcw9R9xc',),
 ('خانم زینب اسماعیلی خبرنگار سابق روزنامه جاسوس پرور شرق ، این جمعیت میلیونی که میبینی مردم ایران 🇮🇷 هستند ، اومدن نمایش تراژیک و عذاب آور ببیند!!!!!!\nشرم بر تو 👌 https://t.co/JiUnVKOXzK',),
 ('هم\u200cاکنون؛ تصویری از رهبر انقلاب در ابتدای مراسم عزاداری شام شهادت حضرت فاطمه\u200cزهرا سلام\u200cالله\u200cعلیها در حسینیه امام خمینی(ره). ۱۴۰۲/۰۹/۲۶ https://t.co/Z3AZqJsY5f',),
 ('به كجا چنين شتابان. ، سردار از #آیزنهاور پرسيد😁😁 https://t.co/m24Eb8AyGW',),
 ('هیچ چیز مانند مظلومیت، رسواکننده ظلم و ظالم نیست.\nمنتها یک شرط دارد:\nآن مظلوم باید مقتدر باشد نه ذلیل …\nفاطمه سلام الله علیها بنیانگذار این مکتب است \nبرادران و\u200cخواهرانمان در غزه خواه بدانند یا ندانند پیرو این مکتب هستند.\n\n#مقتدر_مظلوم \n#غزه_تقاوم',),
 ('یا زیاد بخوانید یا اصلا نخوانید!\nکسانی که چند کتاب محدود خوانده\u200cاند به متوهم\u200cترین و خطرناک\u200cترین ان

In [ ]:
target_user_tweets_ids = list(set([x for x in target_user[1]]))
target_user_tweets_ids

['1736407359461704039',
 '1736455332614877486',
 '1736307951638323471',
 '1736442052454310094',
 '1736384901635539042',
 '1736503973559906442',
 '1736422396721062257']

In [ ]:
_xhop_users = tweets_rdd.flatMap(lambda x: users_from_tweets(target_user_tweets_ids,x))
_1hop_users = _xhop_users.collect()
# _1hop_users

In [ ]:
_xhop_tweets = users_rdd.flatMap(lambda x: tweets_from_users(_1hop_users,x))
_2hop_tweets = _xhop_tweets.collect()
# _2hop_tweets

In [ ]:
_xhop_users = tweets_rdd.flatMap(lambda x: users_from_tweets(_2hop_tweets,x))
_3hop_users = _xhop_users.collect()
# _3hop_users

In [ ]:
_xhop_tweets = users_rdd.flatMap(lambda x: tweets_from_users(_3hop_users,x))
_4hop_tweets = _xhop_tweets.collect()
# _4hop_tweets

In [ ]:
all_related_users = _1hop_users
# all_related_users = _1hop_users + _3hop_users
len(all_related_users)

1045

In [ ]:
all_related_tweets = _2hop_tweets
# all_related_tweets = _2hop_tweets + _4hop_tweets
len(all_related_tweets)

97398

In [ ]:
users_for_graph_rdd = users_rdd.flatMap(lambda x: users_with_tweets_generator(all_related_users,x))
users_for_graph = users_for_graph_rdd.collect()

In [ ]:
tweets_for_graph_rdd = tweets_rdd.flatMap(lambda x:tweets_with_users_generator(all_related_tweets,x))
tweets_for_graph = tweets_for_graph_rdd.collect()

In [ ]:
len(users_for_graph)

931

In [ ]:
len(tweets_for_graph)

37396

In [ ]:
seen_users_all = random_walk(target_user,users_for_graph,tweets_for_graph,return_prob = 0.8)

In [ ]:
from itertools import groupby
from operator import itemgetter

first = itemgetter(0)
seen_users_cnt = [(k, sum(item[1] for item in tups_to_sum))
        for k, tups_to_sum in groupby(sorted(seen_users_all, key=first), key=first)]

In [ ]:
seen_users_sorted = sorted(seen_users_cnt, key=lambda x: x[1], reverse=True)
top_users = seen_users_sorted[1:20]
top_users

[('952453705', 755),
 ('37050268', 370),
 ('1591547142136627202', 336),
 ('1303599606551531520', 327),
 ('1343961231166734336', 43),
 ('1575260422516662272', 25),
 ('1705256827213041669', 24),
 ('1593256090908532736', 23),
 ('959142611025059842', 22),
 ('1272999611607719942', 21),
 ('1712456826292428800', 21),
 ('1501786831725240321', 19),
 ('1089593176808148997', 18),
 ('1545831035208093696', 18),
 ('1021430809474076674', 17),
 ('1269654925', 17),
 ('1420411453975273474', 17),
 ('1579439373636046848', 17),
 ('1267421104861261826', 16)]

In [ ]:
idx = next((i for i, v in enumerate(users_for_graph) if v[0] == top_users[1][0]), None)
sample_tweets = users_for_graph[idx][1]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(sample_tweets,x))
recommended_tweets.collect()

[('از هوش مصنوعی بینگ خواستم دوتا ایرانی کنار هم ایستاده را بکشد خروجی زیبا بود. https://t.co/r7UmIfrmX5',),
 ('در پی حمله ناجوانمردانه گروهک تروریستی جیش العدل به مقر انتظامی در شهرستان راسک ۱۱ نفر به شهادت رسیدند.',),
 ('بامداد امروز مقر فرماندهی نیروی انتظامی شهرستان راسک-بلوچستان مورد حمله قرار گرفت، بیش از پانزد نفر از نیروهای انتظامی و امنیتی جانباخته و تعدادی نیز زخمی شدند. https://t.co/TAgfZRFjMv',),
 ('تعداد فالور\u200cهای صادق بوقی به بیش از نیم میلیون رسید و همچنان در حال رشد است\n«ظهور اینفلوئنسری دیگر از دل یک بحران سازی» https://t.co/Cp6n28aEDS',),
 ('بازخوانی صحبت های رئیس سازمان پدافند غیرعامل کشور پس از حمله سایبری به جایگاه\u200cهای سوخت در مهر ۱۴۰۱\nحمله سایبری سال گذشته به سامانه سوخت کشور، چند لایه بود. یک لایه این حمله نفوذ فردی بود. لایه دوم آسیب\u200cپذیری زیرساختی بود. لایه سوم قطع خدمت\u200cرسانی به مردم بود.\nآذر ۱۴۰۲ تکرار ماجرا https://t.co/JRiqaMjU1M',),
 ('بازخوانی صحبت های رئیس سازمان پدافند غیرعامل کشور پس از حمله سایبری به جایگاه\u200cهای سوخت در آبان 

In [ ]:
idx = next((i for i, v in enumerate(users_for_graph) if v[0] == top_users[2][0]), None)
sample_tweets = users_for_graph[idx][1]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(sample_tweets,x))
recommended_tweets.collect()

[('@_Mahdiyar313 @sadrabackup قبل از دستکاری نرخ دلار 1001 عامل فسادزا از جمله بانک ها باید اصلاح شود\nکه پس از آن نیازی به دستکاری نرخ ارز نیست\nتقسیم سهمیه بنزین هم نه شدنی است نه منطقی',),
 ('مدت\u200cهاست، رانت\u200cخواران منتظر شب عیدند تا نرخ ارز بالا برود، ولی سیاست تثبیت، مانع شد.\nگفتند، بعد از سال، حتمأ بالا می\u200cرود چون دولت کسری بودجه دارد. ولی دولت با مدیریت ارز در لایحه بودجه 1403، این انتظار را هم شکست.\n\nتلاش رائفی پور برای افزایش نرخ ارز را از این دید هم تحلیل کنید. https://t.co/1YzFtkZnkx',),
 ('حالا که قرار است با نفت تنهاتری (که البته بسیار کار پسندیده ای است ) زیرساخت ها را توسعه دهیم به نظرشما بین توسعه شهر فرودگاهی امام خمینی و یا توسعه ظرفیت های باری \u200c\u200c- مسافری ریلی کشور کدامیک اولویت دارد ؟\n@mehrdadbazrpash',),
 ('هیچکدام از این جنایات فراموش نخواهد شد \nاین بغض مقدسی است که دیر یا زود با انتقام تسلی خواهد یافت',),
 ('در مملکتی که شورای نگهبان مصلحت اندیشی می\u200cکند و مجمع تشخیصِ مصلحت کار فقهی!هیچ چیز درست نخواهد شد .\n#روحانی',),
 ('در مملکتی

***
# $\text{User #5}$

In [ ]:
target_user = users_rdd.takeSample(False,1,887)[0]
target_user

('1237422726555541504',
 ['1736073756659876275',
  '1734630666627658206',
  '1735462996024131735',
  '1736087873684697252',
  '1735352136878174624',
  '1736004428845523278',
  '1733999905402650931',
  '1736073673780433165',
  '1733835920862818716',
  '1733835920862818716',
  '1734326638270361704',
  '1735044953883959694',
  '1735184815471911270',
  '1735033451722760431',
  '1735142567979950502',
  '1735949150758502728',
  '1733894959873314962',
  '1736442716068728985',
  '1736442716068728985',
  '1735549502000427164',
  '1734620988212678686',
  '1735172627592040940',
  '1734870226150592770',
  '1735032338185093549',
  '1735734663228924390',
  '1733963088133509235',
  '1736047058551787968',
  '1736786936893751338',
  '1734993135418101778',
  '1736669204416184695',
  '1735319829005865090',
  '1735937766062793176',
  '1735345878724997389',
  '1735281549610733768',
  '1734089138377576770',
  '1736080272318214253',
  '1736442716068728985'])

In [ ]:
target_user_tweets_ids = list(set([x for x in target_user[1]]))
target_user_tweets_ids

['1736080272318214253',
 '1735949150758502728',
 '1736087873684697252',
 '1735937766062793176',
 '1734993135418101778',
 '1736073673780433165',
 '1733894959873314962',
 '1736669204416184695',
 '1734870226150592770',
 '1736442716068728985',
 '1735032338185093549',
 '1735462996024131735',
 '1734620988212678686',
 '1735549502000427164',
 '1735033451722760431',
 '1736786936893751338',
 '1734089138377576770',
 '1735345878724997389',
 '1734630666627658206',
 '1736004428845523278',
 '1735281549610733768',
 '1733999905402650931',
 '1735044953883959694',
 '1735172627592040940',
 '1736047058551787968',
 '1735142567979950502',
 '1735184815471911270',
 '1735352136878174624',
 '1733835920862818716',
 '1735734663228924390',
 '1735319829005865090',
 '1736073756659876275',
 '1734326638270361704',
 '1733963088133509235']

In [ ]:
target_user_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(target_user_tweets_ids,x))
target_user_tweets.collect()

[('همراه شو عزیز. این درد مشترک هرگز جدا جدا درمان  نمیشود \u200c…\n\n«یکی ورزشکار است و دیگری هنرمند , یکی معترض و یکی فعال حقوق بشر.\nسالها حقوق ما زنان ایرانی در عرصه\u200cهای مختلف توسط جمهوری اسلامی نقض شد و حالا دور از وطن کنار هم ایستادیم و میدانیم قدرت ما در این هم\u200cصحبتی و اتحاد است.» #الهه_توکلیان\n\n#UnitedAgainstIRregime \n\n@MitraHejazipour \n@MojganShajarian \n@MarziehHamidi \n@shimababaeii \n@mariam_claren',),
 ('🔺#نرگس_محمدی در پیامی به پدر و مادر مهسا امینی: حصار استبداد، صدای ما را قدرتمندتر خواهد کرد\n\n▪محمدی در پیامی به پدر و مادر #مهسا_امینی، از اهدای جایزه ساخاروف به او استقبال کرد و گفت که این جایزه صدای مردم ایران را برای آزادی و برابری، بلندتر خواهد کرد\n\nhttps://t.co/mPjeOtSrrV https://t.co/V6OI9ZqO9Z',),
 ('دلیل حملات اوباش امنیتی پهلوی به گلشیفته در فضای مجازی آواز خواندن او نبود، گفتگوی کوتاه او با مهتدی بود که گفت "ما همیشه و تا ابد متحد و هم پیمان هستیم."\nجمهوری اسلامی از اتحاد ما هراسان است.\n#زن_زندگى_آزادى \n#IRGCterrorists\u200c https://t.co/PM

In [ ]:
_xhop_users = tweets_rdd.flatMap(lambda x: users_from_tweets(target_user_tweets_ids,x))
_1hop_users = _xhop_users.collect()
# _1hop_users

In [ ]:
_xhop_tweets = users_rdd.flatMap(lambda x: tweets_from_users(_1hop_users,x))
_2hop_tweets = _xhop_tweets.collect()
# _2hop_tweets

In [ ]:
_xhop_users = tweets_rdd.flatMap(lambda x: users_from_tweets(_2hop_tweets,x))
_3hop_users = _xhop_users.collect()
# _3hop_users

In [ ]:
_xhop_tweets = users_rdd.flatMap(lambda x: tweets_from_users(_3hop_users,x))
_4hop_tweets = _xhop_tweets.collect()
# _4hop_tweets

In [ ]:
all_related_users = _1hop_users
# all_related_users = _1hop_users + _3hop_users
len(all_related_users)

4290

In [ ]:
all_related_tweets = _2hop_tweets
# all_related_tweets = _2hop_tweets + _4hop_tweets
len(all_related_tweets)

272437

In [ ]:
users_for_graph_rdd = users_rdd.flatMap(lambda x: users_with_tweets_generator(all_related_users,x))
users_for_graph = users_for_graph_rdd.collect()

In [ ]:
tweets_for_graph_rdd = tweets_rdd.flatMap(lambda x:tweets_with_users_generator(all_related_tweets,x))
tweets_for_graph = tweets_for_graph_rdd.collect()

In [ ]:
len(users_for_graph)

3447

In [ ]:
len(tweets_for_graph)

98609

In [ ]:
seen_users_all = random_walk(target_user,users_for_graph,tweets_for_graph,return_prob = 0.8)

In [ ]:
from itertools import groupby
from operator import itemgetter

first = itemgetter(0)
seen_users_cnt = [(k, sum(item[1] for item in tups_to_sum))
        for k, tups_to_sum in groupby(sorted(seen_users_all, key=first), key=first)]

In [ ]:
seen_users_sorted = sorted(seen_users_cnt, key=lambda x: x[1], reverse=True)
top_users = seen_users_sorted[1:20]
top_users

[('728252911675944961', 333),
 ('790728', 283),
 ('1955124895', 235),
 ('410718052', 211),
 ('1378970222', 146),
 ('947924373029171200', 134),
 ('1149790135073947648', 133),
 ('1532043452761919488', 88),
 ('40535534', 83),
 ('1596294391', 80),
 ('47448563', 70),
 ('1002149552974106624', 69),
 ('1074665434442596352', 69),
 ('1362367238980964355', 69),
 ('1368752533', 69),
 ('1602916310312173569', 68),
 ('1263089632696250371', 67),
 ('1213894188695085057', 57),
 ('1554223232575840259', 56)]

In [ ]:
idx = next((i for i, v in enumerate(users_for_graph) if v[0] == top_users[5][0]), None)
sample_tweets = users_for_graph[idx][1]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(sample_tweets,x))
recommended_tweets.collect()

[('مسیح علی\u200cنژاد با اشاره به نامه اخیر دو سناتور آمریکایی به وزیر خارجه آمریکا درباره تهدیدها علیه این فعال سیاسی، خواستار حمایت آمریکا از فعالان دموکراسی\u200cخواه شد. او همچنین برای دیدار با وزیر خارجه آمریکا و صحبت درباره راه\u200cکارهای محافظت از شهروندان آمریکایی اعلام آمادگی کرد. https://t.co/gwA2NYn3Mo',),
 ('مامور حجاب به زن شجاعی که از آزارگری و مزاحمت\u200cشان فیلم می\u200cگیرد میگوید: «بفرست برای مسیح علینژاد، هیچ غلطی نتونست بکنه» \n\nهمین که یک زن آگاه در مقابل گله\u200cای از محجبه\u200cهای حکومتی شجاعانه می\u200cایستد و شرمسارتان می\u200cکند نشان می\u200cدهد که شما غلطی نمی\u200cتوانید بکنید.\n\nبا این همه دوربین و\u200cبگیر و ببند و کشتار، نتوانستید حریف زنان شجاع ایران شوید که توی صورت\u200cشما می\u200cایستند و به برده\u200cداری اسلامی «نه» می\u200cگویند.\n\nحق دارید از ما عصبانی باشید. چون استخوان گلوی حکومت شد\u200cه\u200cایم و نمگیذاریم آب خوش از گلوی نظاممقدس\u200cتان  پایین برود. چون خامنه\u200cای به شما دستور داد جلوی خراب شدن دیوار برلین جمهوری اسلامی بایستی

In [ ]:
idx = next((i for i, v in enumerate(users_for_graph) if v[0] == top_users[2][0]), None)
sample_tweets = users_for_graph[idx][1]

In [ ]:
recommended_tweets = parsed_rdd.flatMap(lambda x: tweet_parser(sample_tweets,x))
recommended_tweets.collect()

[('هیاتی متشکل از ایرانیان در تبعید یا به تعبیری دیگر «دیاسپورای ایرانی با نام-ژینا-مهسا امینی در محکومیت حضور نمایندگان جمهوری اسلامی در مجمع جهانی پناهندگان سازمان ملل» واقع در شهر ژنو، فردا و همزمان با سفر مقامات جمهوری اسلامی، یک نشست خبری برگزار خواهد کرد.\n\nفردا ۱۳ دسامبر یک هیات از زنان دیاسپورای ایرانی شامل وکلای بین المللی، مدافعان حقوق بشر، کارشناسان ژئوپلتیک، شماری از چهره های معروف و نیز قربانیان رژیم آپارتاید جمهوری اسلامی ایران ساعت یک بعدازظهر به وقت محلی نشستی برگزار خواهند کرد. \nاین گروه هدف عمده و اصلی این گروه برای برگزاری چنین نشستی، محکوم کردن نیت رژیم در  سوء استفاده از مجامع جهانی سازمان ملل با هدف مشروعیت زدایی از ارزشهای حقوق بشری و نیز از بین بردن اعتماد عمومی نسبت به سازمان ملل است. \nرژیم جمهوری اسلامی از این مجمع جهانی پناهندگان سازمان ملل با هدف ارائه اطلاعاتی نادرست و اغراق آمیز به منظور گمراه کردن توجه سازمان ملل درباره نقض حقوق بشر در ایران سوء استفاده می کند.  \n\nبی شک رژیم از نشست مجمع جهانی در امور پناهندگان از افکار عمومی به نفع خودش بهره برداری 